# How to Aggregate and Plot AmpTools .fit Results
If you're here, you've most likely just finished running an amplitude analysis using the [AmpTools](https://github.com/mashephe/AmpTools) framework, and now you have a whole bunch of fit results, stored as `.fit` files, that you need to start plotting. Within this tutorial you will learn how to:
1. Aggregate these fit and data files into flattened `.csv` files
2. Load and plot the `.csv` fit results using python's pandas and matplotlib libraries
   1. Analyze mass independent fit results across multiple bins of invariant mass
   1. Analyze the distributions of several fit results within a mass bin

As mentioned in the [repo's README](../README.md), this tutorial assumes you are already familiar with how AmpTools functions and conducts an amplitude analysis. Please note that this tutorial will *not* cover plotting the fit's angular distributions. This requires the full information of the `.root` files and is effectively handled by the [halld_sim plotter scripts](https://github.com/JeffersonLab/halld_sim/blob/master/src/programs/AmplitudeAnalysis/vecps_plotter/vecps_plotter.cc)

Without further ado, lets get started!

## File Aggregation
We will want to create the following 2 files in preparation for our analysis:
1. `data.csv`: constructed from the `.root` data files in each mass bin, containing the information for that bin
2. `best_fits.csv`: contains all the fit results across the entire mass range, made from the "best" of all the randomized fits in each bin

Before getting started, lets get the parent path of this repo for convenient file loading.

In [1]:
from pathlib import Path
parent_dir = str(Path().resolve().parents[0]) # 0 is the 1st parent directory, i.e. the repo home
print(parent_dir)

import sys
sys.path.insert(0, parent_dir) # add the repo home directory to the list of directories Python uses to look for modules

/w/halld-scshelf2101/kscheuer/PyAmpPlots


### Data
If we want to plot our fit results, we need to include the original data we are actually fitting to. This information is unfortunately not included in the `.fit` files, so we need to read it into a `data.csv` file using [convert_to_csv.py](../scripts/convert_to_csv.py). These python scripts use `argparse`, so we can conveniently see its abilities through its help message

In [2]:
%run -i $parent_dir/scripts/convert_to_csv.py -h

usage: convert_to_csv.py [-h] [-t {fit,root}] [-i INPUT [INPUT ...]]
                         [-s SORTED] [-o OUTPUT] [-a ACCEPTANCE_CORRECTED]
                         [-p]

This script converts AmpTools .fit file(s) or its associated ROOT files into a
csv. This script is used for two fit result purposes: 1. To aggregate the
AmpTools .fit files into a single .csv file for easier analysis. 2. To convert
the ROOT files that the .fit files are based off of into a .csv file. Behind
the scenes, this script calls a ROOT macro for either situation.

optional arguments:
  -h, --help            show this help message and exit
  -t {fit,root}, --type {fit,root}
                        Type of file to convert. Can be either 'fit' for
                        AmpTools .fit files, or 'root' for the ROOT data files
                        that the .fit files are based off of
  -i INPUT [INPUT ...], --input INPUT [INPUT ...]
                        Input .fit file(s). Also accepts path(s) with a
    

TODO: complete these steps by explaining how to combine data files

### Fit Results
Now we will be combining all our `.fit` results across the mass bins into a flattened `.csv` file to prepare them for analysis via python. This is achieved again through the [convert_to_csv.py](../scripts/convert_to_csv.py) script, which behind the scenes interacts with [extract_fit_results.cc](../scripts/extract_fit_results.cc) to load the AmpTools `FitResults` class and import the information we need.

TODO: complete steps for loading in just the fit files. the Rand fit analysis can be done in its own section

## Analysis

### Mass independent fit results

### Randomized fit results

TODO: describe here how we can aggregate a csv for just a single mass bin

NOTE: Aggregation need not be strictly applied to mass bin results. Bootstrap, $-t$ bins, or any other collection of `.fit` results can be grouped together.